In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import json
import seaborn as sns
import gc

## Importing datasets

In [2]:
product_dtypes = {'product':'uint16','product_category':'object','product_subcategory': 'object',
                  'var_1': 'float16','var_2': 'float16','var_3': 'float16','var_4':'float16',
                  'var_5': 'float16','var_6': 'float16','var_7': 'float16','var_8':'float16',
                  'var_9':'float16','var_10':'float16'}

In [3]:
city_dict={}
f = open('city_dict.json')
city = json.load(f)
for key in city.keys():
    city_dict[int(key)]=city[key]
f.close()
del(city)
#city_dict

In [4]:
products = pd.read_csv('product_information.csv',dtype=product_dtypes)
#foot_fall = pd.read_csv('foot_fall.csv')
h_disc = pd.read_csv('historical_discount.csv')
e_disc = pd.read_csv('expected_discount.csv')

In [5]:
sales_15 = pd.read_csv('2015_sales_data.csv')
sales_16 = pd.read_csv('2016_sales_data.csv')
sales_17 = pd.read_csv('2017_sales_data.csv')
sales_18 = pd.read_csv('2018_sales_data.csv')

df=[sales_15,sales_16, sales_17, sales_18]
sales = pd.concat(df, keys = ['2015','2016','2017','2018'])

del(sales_15,sales_16,sales_17,sales_18)

## Resetting dtypes and reducing memory usage

In [6]:
sales.date=pd.to_datetime(sales.date)
sales['city']=sales['city'].astype(np.uint8)
sales['product']=sales['product'].astype(np.uint16)
sales['sales']=sales['sales'].astype(np.int64)

In [7]:
cities=['Ahmedabad', 'Bangalore', 'Chennai', 'Delhi', 'Hyderabad',
       'Jaipur', 'Kolkata', 'Lucknow', 'Mumbai', 'Other']

'''
foot_fall.set_index('city', inplace=True)
foot_fall=foot_fall.T.reset_index()
foot_fall.rename(columns={'index':'date'},inplace=True)
foot_fall.date = pd.to_datetime(foot_fall.date)

foot_fall[cities]=foot_fall[cities].fillna(0).astype(np.uint16)
'''

"\nfoot_fall.set_index('city', inplace=True)\nfoot_fall=foot_fall.T.reset_index()\nfoot_fall.rename(columns={'index':'date'},inplace=True)\nfoot_fall.date = pd.to_datetime(foot_fall.date)\n\nfoot_fall[cities]=foot_fall[cities].fillna(0).astype(np.uint16)\n"

In [8]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3389 entries, 0 to 3388
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   product              3389 non-null   uint16 
 1   product_category     3389 non-null   object 
 2   product_subcategory  3389 non-null   object 
 3   var_1                3389 non-null   float16
 4   var_2                3389 non-null   float16
 5   var_3                3389 non-null   float16
 6   var_4                3389 non-null   float16
 7   var_5                3389 non-null   float16
 8   var_6                3389 non-null   float16
 9   var_7                3389 non-null   float16
 10  var_8                3389 non-null   float16
 11  var_9                3389 non-null   float16
 12  var_10               3389 non-null   float16
dtypes: float16(10), object(2), uint16(1)
memory usage: 125.9+ KB


In [9]:
products = products.replace({'^Cat_':'','SubCat_':''}, regex =True)
products[['product_category', 'product_subcategory']] = products[['product_category', 
                                                                  'product_subcategory']].apply(pd.to_numeric)
products[['product_category', 'product_subcategory']]=products[['product_category', 'product_subcategory']].astype(np.uint16)

In [10]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3389 entries, 0 to 3388
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   product              3389 non-null   uint16 
 1   product_category     3389 non-null   uint16 
 2   product_subcategory  3389 non-null   uint16 
 3   var_1                3389 non-null   float16
 4   var_2                3389 non-null   float16
 5   var_3                3389 non-null   float16
 6   var_4                3389 non-null   float16
 7   var_5                3389 non-null   float16
 8   var_6                3389 non-null   float16
 9   var_7                3389 non-null   float16
 10  var_8                3389 non-null   float16
 11  var_9                3389 non-null   float16
 12  var_10               3389 non-null   float16
dtypes: float16(10), uint16(3)
memory usage: 86.2 KB


In [11]:
h_disc.rename(columns={'Discount_Ahmedabad':'Ahmedabad', 'Discount_Bangalore':'Bangalore',
                       'Discount_Chennai':'Chennai', 'Discount_Delhi':'Delhi', 'Discount_Hyderabad':'Hyderabad',
                       'Discount_Jaipur':'Jaipur', 'Discount_Kolkata':'Kolkata', 'Discount_Lucknow':'Lucknow',
                       'Discount_Mumbai':'Mumbai', 'Discount_Other':'Other'}, inplace=True)

h_disc[cities]=h_disc[cities].astype(np.uint8)
h_disc['product']=h_disc['product'].astype(np.uint16)
h_disc.date=pd.to_datetime(h_disc.date)

h_disc=(h_disc.melt(['date','product'], 
             var_name='city', 
             value_name='discount'))

#h_disc.info()

In [12]:
e_disc.rename(columns={'Discount_Ahmedabad':'Ahmedabad', 'Discount_Bangalore':'Bangalore',
                       'Discount_Chennai':'Chennai', 'Discount_Delhi':'Delhi', 'Discount_Hyderabad':'Hyderabad',
                       'Discount_Jaipur':'Jaipur', 'Discount_Kolkata':'Kolkata', 'Discount_Lucknow':'Lucknow',
                       'Discount_Mumbai':'Mumbai', 'Discount_Other':'Other'}, inplace=True)

e_disc[cities]=e_disc[cities].astype(np.uint8)
e_disc['product']=e_disc['product'].astype(np.uint16)
e_disc.date=pd.to_datetime(e_disc.date)

e_disc=(e_disc.melt(['date','product'], 
             var_name='city', 
             value_name='discount'))
#e_disc.info()

In [13]:
gc.collect()

60

## Data wrangling

In [14]:
print(city_dict)
#print('foot fall',foot_fall.shape)
print('products',products.shape)
print('Historical discounts',h_disc.shape)
print('expected discount',e_disc.shape)

{4: 'Mumbai', 5: 'Delhi', 7: 'Bangalore', 1: 'Kolkata', 6: 'Chennai', 8: 'Hyderabad', 9: 'Ahmedabad', 3: 'Jaipur', 10: 'Lucknow', 2: 'Other'}
products (3389, 13)
Historical discounts (1127530, 4)
expected discount (309870, 4)


In [15]:
#Replacing city ids with names
def cv(x):
    return (city_dict[x])
sales.city = sales.city.apply(cv)

#Deleting duplicate entries
print('old shape',sales.shape)
sales.drop_duplicates(['date','product','city'], inplace=True)
sales.reset_index(drop=True,inplace=True)
print('new shape',sales.shape)

old shape (21135745, 4)
new shape (21113141, 4)


In [16]:
# Removing outliers
sales=sales[(sales['sales']>-2000)]
sales = sales[sales['sales']<5000]
sales.reset_index(drop=True,inplace=True)
sales.shape

(21112679, 4)

## Working with features, merging datasets

In [17]:
result = pd.merge(sales, h_disc, how='left', on=['date', 'product','city'] ).fillna(0)

In [18]:
del(sales, h_disc)

gc.collect()

20

In [19]:
#Converting categorical cities to city ids
new_dict = dict([(value, key) for key, value in city_dict.items()]) 
result.replace(new_dict, inplace=True)

In [20]:
features = pd.merge(result, products,how='left', on=['product']).fillna(0)

In [21]:
del(result,products)

gc.collect()

40

In [22]:
#features.info()
features['day'] = features['date'].map(lambda x: x.day)
features['month'] = features['date'].map(lambda x: x.month)
features['year'] = features['date'].map(lambda x: x.year)
features.drop('date', axis=1, inplace=True)


In [23]:
features.reset_index(drop=True,inplace=True)

In [24]:
gc.collect()

40

In [25]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21112679 entries, 0 to 21112678
Data columns (total 19 columns):
 #   Column               Dtype  
---  ------               -----  
 0   city                 int64  
 1   product              uint16 
 2   sales                int64  
 3   discount             float64
 4   product_category     uint16 
 5   product_subcategory  uint16 
 6   var_1                float16
 7   var_2                float16
 8   var_3                float16
 9   var_4                float16
 10  var_5                float16
 11  var_6                float16
 12  var_7                float16
 13  var_8                float16
 14  var_9                float16
 15  var_10               float16
 16  day                  int64  
 17  month                int64  
 18  year                 int64  
dtypes: float16(10), float64(1), int64(5), uint16(3)
memory usage: 1.5 GB


In [26]:
features.columns

Index(['city', 'product', 'sales', 'discount', 'product_category',
       'product_subcategory', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5',
       'var_6', 'var_7', 'var_8', 'var_9', 'var_10', 'day', 'month', 'year'],
      dtype='object')

In [27]:
dtypes = {'city':'uint8','discount':'uint8', 'product_category':'uint16',
       'product_subcategory':'uint16', 'day':'uint8', 'month':'uint8', 'year':'uint16'}
c = ['city', 'discount', 'product_category','product_subcategory', 'day', 'month', 'year']
features[c]=features[c].astype(dtypes)

In [28]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21112679 entries, 0 to 21112678
Data columns (total 19 columns):
 #   Column               Dtype  
---  ------               -----  
 0   city                 uint8  
 1   product              uint16 
 2   sales                int64  
 3   discount             uint8  
 4   product_category     uint16 
 5   product_subcategory  uint16 
 6   var_1                float16
 7   var_2                float16
 8   var_3                float16
 9   var_4                float16
 10  var_5                float16
 11  var_6                float16
 12  var_7                float16
 13  var_8                float16
 14  var_9                float16
 15  var_10               float16
 16  day                  uint8  
 17  month                uint8  
 18  year                 uint16 
dtypes: float16(10), int64(1), uint16(4), uint8(4)
memory usage: 805.4 MB


In [29]:
y = features['sales']
features.drop('sales', axis=1, inplace=True)

## Test data

In [ ]:
test_ = pd.read_csv('test_data.csv')
sample_ = pd.read_csv('sample_submission.csv')
test_.head()

In [ ]:
test_.city.value_counts()

In [ ]:
m=test_['product'].isin(features['product'])
m.value_counts()

In [ ]:
test_.info()

In [ ]:
# Dtypes
test_.date=pd.to_datetime(test_.date)
test_.id = test_.id.astype(np.uint64)
test_['city']=test_['city'].astype(np.uint8)
test_['product']=test_['product'].astype(np.uint16)
#df['sales']=df['sales'].astype(np.int64)
    
# City ids to names
test_.city = test_.city.apply(cv)


In [ ]:
# Merging
df1 = pd.merge(test_, e_disc, how='left', on=['date', 'product','city'] ).fillna(0)
del e_disc,test_
    
# Changing back city names to ids
df1=df1.replace(new_dict)
    
# Uploading products data
products = pd.read_csv('product_information.csv',dtype=product_dtypes)
products = products.replace({'^Cat_':'','SubCat_':''}, regex =True)
products[['product_category', 'product_subcategory']] = products[['product_category',
                                                                      'product_subcategory']].apply(pd.to_numeric)
products[['product_category', 'product_subcategory']]=products[['product_category', 'product_subcategory']].astype(np.uint16)
    
# Merging
df2 = pd.merge(df1, products,how='left', on=['product']).fillna(0)
del (df1,products)
    
gc.collect()
    
df2['day'] = df2['date'].map(lambda x: x.day)
df2['month'] = df2['date'].map(lambda x: x.month)
df2['year'] = df2['date'].map(lambda x: x.year)
df2.drop('date', axis=1, inplace=True)
df2.reset_index(drop=True,inplace=True)
        
gc.collect()

In [ ]:
gc.collect()

In [ ]:
df2.info()

In [ ]:
df2.set_index('id',inplace=True)

In [ ]:
dtypes = {'city':'uint8','discount':'uint8', 'product_category':'uint16',
       'product_subcategory':'uint16', 'day':'uint8', 'month':'uint8', 'year':'uint16'}
c = ['city', 'discount', 'product_category','product_subcategory', 'day', 'month', 'year']
df2[c]=df2[c].astype(dtypes)

### **Normalizing data

In [ ]:
'''
cols = ['city', 'product','product_category','discount',
        'product_subcategory', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5',
        'var_6', 'var_7', 'var_8', 'var_9', 'var_10','day', 'month', 'year']
from sklearn import preprocessing
features[cols] = preprocessing.normalize(features[cols])
del (features)